In [7]:
from requests_html import HTMLSession, AsyncHTMLSession
import pandas as pd

# Create a new HTMLSession object
s = HTMLSession()

# Initialize an empty list to store the product data
product_list = []


In [8]:
def parse(products):
    for item in products.absolute_links:
        # Send a GET request to the product URL
        r = s.get(item)
        
        # Extract the product name, subtext, and price using appropriate HTML selectors
        name = r.html.find('div.product-detail-info-title', first=True).text
        subtext = r.html.find('div.product-subtext', first=True).text
        price = r.html.find('span.price', first=True).text

        # Determine the stock status based on the presence of the add-to-cart container
        if r.html.find('div.add-to-cart-container'):
            stock = 'In Stock'
        else:
            stock = 'Out of Stock'
        
        # Create a dictionary to store the product data
        product_dict = {
            'product_name': name,
            'product_price': price,
            'product_description': subtext,
            'product_stock': stock
        }
        
        # Append the product dictionary to the product_list
        product_list.append(product_dict)


In [9]:
asession = AsyncHTMLSession()

for x in range(1, 5):
    url = f"https://www.beerwulf.com/en-gb/c/all-beers?segment=Beers&catalogCode=Beer_1&routeQuery=all-beers&page={x}"
    
    # Send a GET request to the URL using the AsyncHTMLSession
    r = await asession.get(url)
    
    # Wait for the page to render using await
    await r.html.arender()
    
    # Extract the raw HTML content
    resp = r.html.raw_html
    
    # Extract the products using an XPath selector
    products = r.html.xpath('//*[@id="product-items-container"]', first=True)
    
    if products:
        # Parse the product data
        parse(products)


In [10]:
df = pd.DataFrame(product_list)
df

,product_name,product_price,product_description,product_stock
0,Affligem Blond - 5L Draught Keg,£ 26.99,Blond Beer | 6.7% | 5 Litres,In Stock
1,Newcastle Brown Ale - 2L SUB Keg,£ 11.29,Brown Ale | 4.7% | 2 Litres,In Stock
2,Wiener Lager - 8L BLADE Keg,£ 37.99,Lager | 5.5% | 8 Litres,In Stock
3,Puntigamer - 2L SUB Keg,£ 10.99,Lager | 5.2% | 2 Litres,In Stock
4,Delirium Tremens - 2L SUB Keg,£ 18.99,Blond Beer | 8.5% | 2 Litres,In Stock
...,...,...,...,...
87,Amstel - 2L SUB Keg,£ 9.59,Lager | 5.0% | 2 Litres,In Stock
88,Fruh Kölsch - 2L SUB Keg,£ 13.99,Lager | 4.8% | 2 Litres,In Stock
89,Pelforth Blonde - 2L SUB Keg,£ 10.99,Blond Beer | 5.8% | 2 Litres,In Stock
90,Gebrouwen Door Vrouwen Bloesem Blond,£ 49.99,Blond Beer | 6.2% | 33 cl,Out of Stock
